# Chap06 - 텍스트 2: 단어 벡터, 고급 RNN, 임베딩 시각화

> [5장](http://excelsior-cjh.tistory.com/154)에서 살펴본 텍스트 시퀀스를 좀 더 깊이 알아보며, **word2vec**이라는 비지도학습 방법을 사용하여 단어 벡터를 학습하는 방법과 텐서보드를 사용해서 임베딩을 시각화 하는 방법에 대해 알아보자. 그리고 RNN의 업그레이드 버전인 **GRU**에 대해서 알아보자.

## 6.1 단어 임베딩 소개

[5.3.2](http://excelsior-cjh.tistory.com/154)에서 텐서플로를 이용해 텍스트 시퀀스를 다루는 방법을 알아 보았다. 단어 ID를 저차원의 Dense vector로의 매핑을 통해 단어 벡터를 학습시켰다. 이러한 처리가 필요한 이유는 RNN의 입력으로 넣어 주기 위해서였다.

> TensorFlow is an open source software library for high performance numerical computation.

위의 문장을 [5.3.2](http://excelsior-cjh.tistory.com/154)에서처럼, 각 단어를 ID로 표현 한다면 'tensorflow'는 정수 2049에, 'source'라는 단어는 17, 'performance'는 0으로 매핑할 수 있다.

하지만, 이러한 방법은 몇 가지 문제점이 있다. 

- 단어의 의미를 잃어버리게 되고, 단어 사이의 의미론적 근접성(semantic proximity)과 관련 정보를 놓치게 된다. 예를 들어, 위의 문장에서 'high'와 'software'는 서로 관련이 없지만, 이러한 정보는 반영되지 않는다.
- 단어의 수가 엄청 많을 경우 단어당 ID개수 또한 많아지게 되므로 단어의 벡터 표현이 희소(sparse)해져 학습이 어려워진다.

이러한 문제를 해결하기 위한 방법 중 하나는 비지도학습(Unsupervised Learning)인 word2vec을 이용하는 것이다. 이 방법의 핵심은 **분포 가설(Distributional Hypothesis)**이며, 언어학자 존 루퍼트 퍼스(John Rupert Firth)가 한 유명한 말로 설명할 수 있다.

> "You shall know a word by the company it keeps."- "단어는 포함된 문맥 속에서 이해할 수 있다."

즉, 비슷한 맥락에서 함께 나타나는 경향이 있는 단어들은 비슷한 의미를 가지는 경향이 있다.

## 6.2 word2vec

